In [42]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

## Data

### Read Data

In [43]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [44]:
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


In [45]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



### Tokenization

In [46]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [47]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [48]:
print(encode("Helllo! What's up?"))
print(decode([20, 43, 50, 50, 50, 53, 2, 1, 35, 46, 39, 58, 5, 57, 1, 59, 54, 12]
))

[20, 43, 50, 50, 50, 53, 2, 1, 35, 46, 39, 58, 5, 57, 1, 59, 54, 12]
Helllo! What's up?


In [49]:
data = torch.tensor(encode(text), dtype=torch.long)

In [50]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


### Train/Val Split

In [51]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

### Data Loader

In [52]:
block_size = 8
train_data[: block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [53]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Input:{context}, Target:{target}")

Input:tensor([18]), Target:47
Input:tensor([18, 47]), Target:56
Input:tensor([18, 47, 56]), Target:57
Input:tensor([18, 47, 56, 57]), Target:58
Input:tensor([18, 47, 56, 57, 58]), Target:1
Input:tensor([18, 47, 56, 57, 58,  1]), Target:15
Input:tensor([18, 47, 56, 57, 58,  1, 15]), Target:47
Input:tensor([18, 47, 56, 57, 58,  1, 15, 47]), Target:58


In [54]:
torch.manual_seed(1337)
batch_size = 4 # how many independdent sequences will we process in parallel
block_size = 8 # what is max context length for predictions

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
  for t in range(block_size):
      context = xb[b, :t+1]
      target = yb[b, t]
      print(f"Input:{context.tolist()}, Target:{target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
Input:[24], Target:43
Input:[24, 43], Target:58
Input:[24, 43, 58], Target:5
Input:[24, 43, 58, 5], Target:57
Input:[24, 43, 58, 5, 57], Target:1
Input:[24, 43, 58, 5, 57, 1], Target:46
Input:[24, 43, 58, 5, 57, 1, 46], Target:43
Input:[24, 43, 58, 5, 57, 1, 46, 43], Target:39
Input:[44], Target:53
Input:[44, 53], Target:56
Input:[44, 53, 56], Target:1
Input:[44, 53, 56, 1], Target:58
Input:[44, 53, 56, 1, 58], Target:46
Input:[44, 53, 56, 1, 58, 46], Target:39
Input:[44, 53, 56, 1, 58, 46, 39], Target:58
Input:[44, 53, 56, 1, 58, 46, 39, 58], Target:1
Input:[52], Target:58
Input:[52, 58], Target

## NN

In [57]:
class BigramLM(nn.Module):
    
    def __init__(self, vocab_size):
      super().__init__()
      self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
      logits = self.token_embedding_table(idx) # (B, T, C)
      B,T,C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      return logits, loss
    
    def generate(self, idx, max_new_tokens):
      # idx is (B,T) array of indices in the current context
      for _ in range(max_new_tokens):
        logits, loss = self(idx) # get predictions
        logits = logits[:, -1, :] # focus only on last time step, becomes (B,C)
        probs = F.softmax(logits, dim=1) # (B,C)
        idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
        idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
      return idx
        

m = BigramLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.7288, grad_fn=<NllLossBackward0>)
